In [2]:
import pandas as pd

<h1>PlayTimeGenre</h1>

def PlayTimeGenre( genero : str ): Debe devolver año con mas horas jugadas para dicho género.
Ejemplo de retorno: {"Año de lanzamiento con más horas jugadas para Género X" : 2013}

In [3]:
# Cargamos los archivos CSV a utilizar:
games_df = pd.read_csv(r"..\Archivos Producidos\games.csv")
users_df = pd.read_csv(r"..\Archivos Producidos\users.csv")

In [4]:
games_df.dtypes

app_name                     object
release_year                  int64
item_id                       int64
developer                    object
Action                        int64
Adventure                     int64
Animation &amp; Modeling      int64
Audio Production              int64
Casual                        int64
Design &amp; Illustration     int64
Early Access                  int64
Education                     int64
Free to Play                  int64
Indie                         int64
Massively Multiplayer         int64
Photo Editing                 int64
RPG                           int64
Racing                        int64
Simulation                    int64
Software Training             int64
Sports                        int64
Strategy                      int64
Utilities                     int64
Video Production              int64
Web Publishing                int64
dtype: object

In [5]:
users_df.dtypes

user_id     object
playtime     int64
item_id      int64
dtype: object

Ya cargados los archivos, comenzamos con las transformaciones:

In [6]:
# Unimos las columnas que nos serán de utilidad:
df_u_g = pd.merge(users_df[["item_id", "playtime"]], games_df[["item_id", "release_year"]], on="item_id")
df_u_g.head()

,item_id,playtime,release_year
0,10,6,2000
1,10,0,2000
2,10,0,2000
3,10,328,2000
4,10,580,2000


In [7]:
# Agrupamos por "item_id" y "release_year" y sumamos las horas jugadas:
df_sum_playtime = df_u_g.groupby(["item_id", "release_year"])["playtime"].sum().reset_index()
df_sum_playtime.head()

,item_id,release_year,playtime
0,10,2000,14585747
1,20,1999,92533
2,30,2003,33393
3,40,2001,789
4,50,1999,91574


In [8]:
# Unimos el dataframe anterior con los dummies de géneros:
df_playtimegenre = pd.merge(df_sum_playtime, games_df[['item_id', 'Action',
       'Adventure', 'Animation &amp; Modeling', 'Audio Production', 'Casual',
       'Design &amp; Illustration', 'Early Access', 'Education',
       'Free to Play', 'Indie', 'Massively Multiplayer', 'Photo Editing',
       'RPG', 'Racing', 'Simulation', 'Software Training', 'Sports',
       'Strategy', 'Utilities', 'Video Production', 'Web Publishing']], on="item_id")

df_playtimegenre.head()

,item_id,release_year,playtime,Action,Adventure,Animation &amp; Modeling,Audio Production,Casual,Design &amp; Illustration,Early Access,...,Photo Editing,RPG,Racing,Simulation,Software Training,Sports,Strategy,Utilities,Video Production,Web Publishing
0,10,2000,14585747,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,20,1999,92533,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,30,2003,33393,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,40,2001,789,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,50,1999,91574,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
# Utilizamos melt para convertir el DataFrame de ancho a largo:
df_melted = pd.melt(df_playtimegenre, id_vars=["item_id", "release_year", "playtime"], 
                    var_name="genre", value_name="has_genre")

# Filtramos las filas donde el género está presente (has_genre == 1):
df_filtered = df_melted[df_melted["has_genre"] == 1]

# Agrupamos por "release_year" y "genre" y sumamos las horas jugadas:
df_playtimegenre_final = df_filtered.groupby(["release_year", "genre"], as_index=False)["playtime"].sum()

# Capitalizamos el nombre del género:
df_playtimegenre_final["genre"] = df_playtimegenre_final["genre"].str.replace('&amp;', '&').apply(str.capitalize)

print(df_playtimegenre_final)

     release_year                  genre  playtime
0            1990                 Action      2935
1            1990              Adventure        52
2            1992              Adventure         0
3            1993               Strategy        33
4            1994                 Action      1649
..            ...                    ...       ...
186          2017  Massively multiplayer    291231
187          2017                    Rpg    292453
188          2017             Simulation    226570
189          2017                 Sports        12
190          2017               Strategy      1222

[191 rows x 3 columns]


In [10]:
# Añadimos a la lista aquellos géneros que no poseen horas jugadas:

# Revisamos todos los géneros existentes:
todos_los_generos_nocap = ['Action',
       'Adventure', 'Animation & Modeling', 'Audio Production', 'Casual',
       'Design & Illustration', 'Early Access', 'Education',
       'Free to Play', 'Indie', 'Massively Multiplayer', 'Photo Editing',
       'RPG', 'Racing', 'Simulation', 'Software Training', 'Sports',
       'Strategy', 'Utilities', 'Video Production', 'Web Publishing']
todos_los_generos_cap = []
for i in todos_los_generos_nocap: # Capitalizamos los géneros
       todos_los_generos_cap.append(i.capitalize())

# Si el género no se encuentra en el dataframe, añadimos un registro:
for genre in todos_los_generos_cap:
       if genre not in df_playtimegenre_final["genre"].tolist():
              df_playtimegenre_final.loc[len(df_playtimegenre_final)] = {"release_year": None, "genre": genre, "playtime": 0}

In [11]:
print(df_playtimegenre_final)

    release_year                  genre  playtime
0           1990                 Action      2935
1           1990              Adventure        52
2           1992              Adventure         0
3           1993               Strategy        33
4           1994                 Action      1649
..           ...                    ...       ...
192         None  Design & illustration         0
193         None              Education         0
194         None          Photo editing         0
195         None      Software training         0
196         None         Web publishing         0

[197 rows x 3 columns]


Desarrollamos la función:

In [12]:
# Creamos la función PlayTimeGenre:
def PlayTimeGenre(genero):

    # Verificamos si la variable introducida es una cadena de texto:
    if type(genero) != str: return f"El género '{genero}' no existe."
    else:

        # Capitalizamos la cadena de texto:
        genero = genero.strip().capitalize()
    
        # Comprobamos si el género existe en el DataFrame
        if genero not in df_playtimegenre_final["genre"].tolist():
            return f"El género '{genero}' no existe."
        
        # # Filtramos las filas para el género específico:
        df_genero = df_playtimegenre_final[df_playtimegenre_final["genre"] == genero]

        # En caso de que el género introducido no tenga horas de juego:
        if (df_genero["playtime"] == 0).any():
            return f"El género '{genero}' no posee horas jugadas."
        
        # En caso de que el género introducido si tenga horas de juego:
        else:

            # Hallamos el índice del registro con mayor cantidad de horas de juego:
            indice_max_playtime = df_genero["playtime"].idxmax()
            
            # Tomamos el año de dicho registro:
            año_max_playtime = df_genero.loc[indice_max_playtime]["release_year"]

            # Devolvemos la respuesta final:
            return {f"Año de lanzamiento con más horas jugadas para Género {genero}" : año_max_playtime}

In [13]:
# Probamos la función
print(PlayTimeGenre("Simulation  "))
print(PlayTimeGenre("acTion"))
print(PlayTimeGenre("Acción"))
print(PlayTimeGenre(123))
print(PlayTimeGenre("Rpg"))
print(PlayTimeGenre("Photo editing"))

{'Año de lanzamiento con más horas jugadas para Género Simulation': 2006}
{'Año de lanzamiento con más horas jugadas para Género Action': 2004}
El género 'Acción' no existe.
El género '123' no existe.
{'Año de lanzamiento con más horas jugadas para Género Rpg': 2011}
El género 'Photo editing' no posee horas jugadas.


Guardamos el DataFrame utilizado:

In [14]:
# Guardamos el dataframe "df_playtimegenre_final":
df_playtimegenre_final.to_csv(r"..\Archivos Producidos\Data para Endpoints\df_playtimegenre_final.csv", index=False)